# Project nr 1.1 Extract
Artjom, Heidi, Kaja, Rasmus

## E1. Fetch data from internet

Original dataset of Scientific Publications is located here: https://originalstatic.aminer.cn/misc/dblp.v13.7z

In [0]:
%sh mkdir data && cd data && wget https://originalstatic.aminer.cn/misc/dblp.v13.7z

--2022-05-13 12:28:22--  https://originalstatic.aminer.cn/misc/dblp.v13.7z
Resolving originalstatic.aminer.cn (originalstatic.aminer.cn)... 159.27.2.14
Connecting to originalstatic.aminer.cn (originalstatic.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2568255035 (2.4G) [application/x-7z-compressed]
Saving to: ‘dblp.v13.7z’

     0K .......... .......... .......... .......... ..........  0%  279K 2h29m
    50K .......... .......... .......... .......... ..........  0%  278K 2h30m
   100K .......... .......... .......... .......... ..........  0%  279K 2h29m
   150K .......... .......... .......... .......... ..........  0%  278K 2h30m
   200K .......... .......... .......... .......... ..........  0%  279K 2h30m
   250K .......... .......... .......... .......... ..........  0%  140K 2h54m
   300K .......... .......... .......... .......... ..........  0%  280K 2h51m
   350K .......... .......... .......... .......... ..........  0%  2

## E2. Unzip downloaded file
- Install 7zip unzipper
- Unzip the compressed file

In [0]:
%sh sudo apt install --assume-yes p7zip-full



Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,545 kB of archives.
After this operation, 5,896 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 p7zip amd64 16.02+dfsg-7build1 [358 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 p7zip-full amd64 16.02+dfsg-7build1 [1,187 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 1,545 kB in 1s (1,693 kB/s)
Selecting previously unselected package p7zip.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 

In [0]:
%sh cd data && 7z x dblp.v13.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz (406F1),ASM,AES-NI)

Scanning the drive for archives:
1 file, 2568255035 bytes (2450 MiB)

Extracting archive: dblp.v13.7z
--
Path = dblp.v13.7z
Type = 7z
Physical Size = 2568255035
Headers Size = 130
Method = LZMA2:24
Solid = -
Blocks = 1

Everything is Ok

Size:       17352640799
Compressed: 2568255035


If all successed then the file **`dblpv13.json`** is in `databricks/driver` folder.

In [0]:
%sh ls data

#%sh ls /databricks/driver/data

dblp.v13.7z
dblpv13.json


## E3. Look into file

In [0]:
%sh head -20 data/dblpv13.json

[
{ 
    "_id" : "53e99784b7602d9701f3e3f5", 
    "title" : "3GIO.", 
    "venue" : {
        "type" : NumberInt(0)
    }, 
    "year" : NumberInt(2011), 
    "keywords" : [

    ], 
    "n_citation" : NumberInt(0), 
    "lang" : "en"
},
{ 
    "_id" : "53e99784b7602d9701f3e133", 
    "title" : "The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.", 
    "authors" : [
        {
            "_id" : "53f45728dabfaec09f209538", 


In [0]:
%sh tail -20 data/dblpv13.json

        "raw" : "SIGGRAPH ASIA Art Gallery & Emerging Technologies"
    }, 
    "year" : 2009.0, 
    "keywords" : [

    ], 
    "n_citation" : 0.0, 
    "page_start" : "39", 
    "lang" : "en", 
    "volume" : "", 
    "issue" : "", 
    "doi" : "10.1145/1665137.1665166", 
    "url" : [
        "http://doi.acm.org/10.1145/1665137.1665166", 
        "db/conf/siggraph/siggraph2009asiaart.html#Palma09", 
        "https://doi.org/10.1145/1665137.1665166"
    ], 
    "abstract" : ""
}
]


## E4. Split source data (and preprocess json)

#### Function for splitting and fixing json

In [0]:
import json
import re
import time

In [0]:
def split(filename,n): # filename to split into pieces with n json objects each
    with open(filename, 'r') as f:
        i=1 # counts json objects
        content='' # for collecting json objects for file
        for row in f:
            if 'NumberInt' in row:  #some rows have NumberInt(55) which is replaced here by 55
                row = re.sub(r"NumberInt\((\d+)\)", r"\1", row)
            content += row
            if (row == '},\n'):# | (row == '}\n'): #criterium pointing to the end of json object
                if i%n == 0:
                    file_name = filename.split('.json')[0] + 'i_' + str(int(i/n)) + '.json'
                    file = open(file_name, 'w')  #save collected json objects
                    file.write(content[:-2]+'\n]')  #we remove comma after last json object in collection
                    file.close()
                    print(file_name+' created')
                    content='[\n' #start collecting from scratch
                i +=1
        file_name = filename.split('.json')[0] + 'i_' + str(int(i/n)+1) + '.json'  # save last piece explicitly
        file = open(file_name, 'w')
        file.write(content)
        file.close()
        print(file_name+' created')
    f.close()

#### Split the data

In [0]:
split('data/dblpv13.json',1e3)

data/dblpv13i_1.json created
data/dblpv13i_2.json created
data/dblpv13i_3.json created
data/dblpv13i_4.json created
data/dblpv13i_5.json created
data/dblpv13i_6.json created
data/dblpv13i_7.json created
data/dblpv13i_8.json created
data/dblpv13i_9.json created
data/dblpv13i_10.json created
data/dblpv13i_11.json created
data/dblpv13i_12.json created
data/dblpv13i_13.json created
data/dblpv13i_14.json created
data/dblpv13i_15.json created
data/dblpv13i_16.json created
data/dblpv13i_17.json created
data/dblpv13i_18.json created
data/dblpv13i_19.json created
data/dblpv13i_20.json created
data/dblpv13i_21.json created
data/dblpv13i_22.json created
data/dblpv13i_23.json created
data/dblpv13i_24.json created
data/dblpv13i_25.json created
data/dblpv13i_26.json created
data/dblpv13i_27.json created
data/dblpv13i_28.json created
data/dblpv13i_29.json created
data/dblpv13i_30.json created
data/dblpv13i_31.json created
data/dblpv13i_32.json created
data/dblpv13i_33.json created
data/dblpv13i_34.js

### Make directories for Warehouse

In [0]:
dbutils.fs.mkdirs("dbfs:/tmp/data")
dbutils.fs.mkdirs("dbfs:/tmp/data/source") # source json-files go there
dbutils.fs.mkdirs("dbfs:/tmp/data/warehouse") # DW lands here

Out[10]: True

In [0]:
tgt = "dbfs:/tmp/data/"
dbutils.fs.ls(tgt)

Out[11]: [FileInfo(path='dbfs:/tmp/data/source/', name='source/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/data/warehouse/', name='warehouse/', size=0, modificationTime=0)]

### Removing DW - keep commented after use!

In [0]:
#dbutils.fs.rm("dbfs:/tmp/data/",recurse=True) # remove
#dbutils.fs.mkdirs("dbfs:/tmp/data") # & create again
#dbutils.fs.mkdirs("dbfs:/tmp/data/source") # source json-files go there
#dbutils.fs.mkdirs("dbfs:/tmp/data/warehouse") # DW lands here